In [1]:
import os

from IPython.display import display, Markdown, Image

In [2]:
REPO_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
TASK_PATH = os.path.join(REPO_PATH, "tasks", "10-vectors.md")
DATA_PATH = os.path.join(os.getcwd(), "1551.gov.ua", "raw")

In [3]:
def show_markdown(path):
    with open(path, 'r') as fh:
        content = fh.read()
    display(Markdown(content))

In [4]:
show_markdown(TASK_PATH)

# Векторні представлення

## Завдання 1

Побудуйте покращений класифікатор, який буде предбачати категорію запиту до служби 1551 з використанням тільки векторів слів (не використовувати інші ознаки, такі як самі слова, нграми і т.д) для всіх категорій.

У якості бейзлайну використайте класифікатор kNN, який ви побудували на практичному.

Зробіть як мінімум 2 ітерації покращення, які можуть полягати у:

- підборі найкращого варіанту векторів з наведених тут: https://lang.org.ua/en/models/#anchor4 або інших (fasttext, Tf-IDF, LDA, ...) Ви навіть можете дотренувати вектори на власній тренувальній множині. Можете також спробувати агрегацію декількох векторів.
- будь-яких видах передпроцесингу і попередньої обробки (фільтрація мови, стопслів, службових частин мови, лематизація, нормалізація, ...)
- побудови векторів документу з використанням таких підходів як pargaraph vectors (doc2vec), ElMo, Universal Sentece Encoder...
- використання довільних алгоритмів класифікації та ансамблів цих алгоритмів
- групуванні категорій (на основі кластеризації або якихось евристик) та побудови багаторівневих класифікаторів

Ваша мета: досягнути такого результату по якості, щоб було не соромно дивитись в очі Кличкові. ;)

## Завдання 2

Візуалізуйте ваші класи з використанням t-SNE. Якщо ви будете використовувати агрегацію категорій, то можете візуалізувати тільки категорії вищого рівня і, для прикладу, одну з них. Якщо ви не будете групувати категорії, то використовуйте для візуалізації найбільш частотні (30-50). Спробуйте досягти достатньо розділеної картинки, експериментуючи з параметрами візуалізації (звісно, якщо ваші вхідні дані дозволять вам це).


## Оцінювання

Оцінка:
- 90 балів за класифікатор (рішення, які будуть мати якість більше 0.8 за macro_average або більше 0.9 за micro_average отримають додаткові 10 балів)
- 10 балів за візуалізацію

Крайній термін: 16.05.2020


In [5]:
import re
import gzip
import json

import pandas as pd
import numpy as np
import pycld2 as cld2

from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from pathlib import Path
from collections import Counter, OrderedDict
from langdetect import detect, lang_detect_exception

In [34]:
def lang_detect(x):
    try:
        return detect(x.lower())
    except lang_detect_exception.LangDetectException:
        return ''

def lang_detect_v2(x):
    try:
        x = ''.join(_ for _ in " ".join(x.split()[:100]).lower() if _.isprintable())
        isReliable, textBytesFound, details = cld2.detect(x)
        res = sorted(details, key=lambda x: x[2], reverse=True)[0][1]
        if res == 'un':
            res = detect(x.lower())
        return res
    except:
        return ""
    
def clean_text(x):
    x = x.lower()
    x = x.replace("”", "’")
    x = " ".join(filter(None, x.split()))
    return x

## Read data

In [9]:
res = []
for file_path in tqdm(Path(DATA_PATH).rglob('*.gz')):
    with gzip.open(file_path, 'rt', encoding='utf8') as zipfile:
        obj = json.load(zipfile)[0]
    res.append(obj)
df = pd.DataFrame(res)

127329it [00:20, 6132.57it/s]


In [10]:
df['manager_name'] = df['CallZManager'].map(lambda x: "" if not re.search(r"\[([\s\w\.]+)\]", x) 
                                                         else re.findall(r"\[([\s\w\.]+)\]", x)[0].strip())
df['entity_name'] = df['CallZManager'].map(lambda x: re.sub(r"\[([\s\w\.]+)\]", "", x))
df['cat_count'] = df['CallZType'].map(df['CallZType'].value_counts().to_dict())

In [35]:
%%time

lst = process_map(lang_detect, df['CallZText'].values, max_workers=8)
df['lang'] = lst


CPU times: user 46.7 s, sys: 7.07 s, total: 53.8 s
Wall time: 2min 10s


In [37]:
print(f"Records with > 100 label samples: {df.loc[df.cat_count > 100].shape[0] / df.shape[0] * 100:.2f} %")

Records with > 100 label samples: 89.95 %


In [38]:
df['lang'].value_counts()

uk    69328
ru    56958
        540
bg      321
mk      169
pl        5
en        2
et        1
sk        1
sw        1
fr        1
sl        1
ca        1
Name: lang, dtype: int64

In [44]:
Counter(" ".join(df['entity_name'].values).lower().split()).most_common(10)

[('кп', 34616),
 ('-', 34534),
 ('жед', 15727),
 ('пат', 13631),
 ('жек', 12826),
 ('”київенерго”', 12645),
 ('відділ', 12027),
 ('трм', 10340),
 ('№', 9781),
 ('управління', 8406)]

In [124]:
data = df.loc[(df.cat_count > 100) & (df.lang.isin(['uk']))][['CallZText', 'CallZType']] # , 'ru'
data.columns = ['text', 'label']
data['text'] = data['text'].map(clean_text)

In [125]:
print(data.shape)
data.head()

(61977, 2)


,text,label
2,доброго дня. прошу відремонтувати бювет за адр...,Не працює бювет
3,просимо відремонтувати вікна в під’їзді №1 або...,Скління та ремонт вікон на сходових клітинах;
6,щодо відповіді від дп «дарниця-1» на моє зверн...,Претензії та зауваження до Начальника
7,добрий день. прошу розібратисячому в будинку в...,Відсутність опалення
9,вынужден обратиться так как в нашем районе мус...,Незручності від вивозу сміття з контейнерів та...


In [126]:
import stanfordnlp
import warnings

warnings.filterwarnings("ignore")

In [54]:
nlp_uk = stanfordnlp.Pipeline(lang='uk')

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_tokenizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_tagger.pt', 'pretrain_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_lemmatizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_parser.pt', 'pretrain_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': '

In [127]:
# for item in data.sample(5).text.values:
#     print(item)
#     print("*********")

незадовільна температура гвп вже декілька днів з горячого крану тече майже холодна вода. прийняті душ неможливо навіть якщо не включати холодну. застосуйте, будь ласка, міри щодо належного постачання послуг гвп.
*********
у будинку здійснювали заміну металопластикових вікон у підїзді. вікно встановлено на 2 поверсі 5 підїзду без створки, що відкривається, тобто зовсім глухе. вікна встановлені в інших підїздах мають по одній створці, що відкриівється на кожному поверсі, але на нашому поверсі відсутня можливість відкривання зовсім. оскільки цей ремонт здійснювався під особистим контролем мера міста киева, то про неякісне виконання ремонту або порушення при цьому було прохання терміново повідомляти. прошу вжити заходів для виправлення ситуациї та встановлення належного вікна з можливостю провітрювання та відкриття для миття.
*********
не відновлено тротуар та не ліквідована яма після розриття київенерго за адресою вул. о.гончара 79, поряд із електрощитком. протягом декількох місяців прямо

In [132]:
string = data.sample(5).text.values[1].lower()
string

'навколо дитячого майданчику біля будинку 34 вул. в. липківського огорожа з навісним замком. немає можливості потрапити на громадський дитячий майданчик. прохання розібратися зі свавіллям мешканців будинку.'

In [133]:
sentences = nlp_uk(string).sentences

In [134]:
for sentence in sentences:
    print([(token.text, token.lemma) for token in sentence.words])
#     print([token for token in sentence.words])
    print()

[('навколо', 'навколо'), ('дитячого', 'дитячий'), ('майданчику', 'майданчик'), ('біля', 'біля'), ('будинку', 'будинок'), ('34', '34'), ('вул', 'вул.'), ('.', '.'), ('в', 'в'), ('.', '.'), ('липківського', 'липківський'), ('огорожа', 'огорожа'), ('з', 'з'), ('навісним', 'навісний'), ('замком', 'замок'), ('.', '.')]

[('немає', 'немати'), ('можливості', 'можливість'), ('потрапити', 'потрапити'), ('на', 'на'), ('громадський', 'громадський'), ('дитячий', 'дитячий'), ('майданчик', 'майданчик'), ('.', '.'), ('прохання', 'прохання'), ('розібратися', 'розібратися'), ('зі', 'зі'), ('свавіллям', 'свавілля'), ('мешканців', 'мешканець'), ('будинку', 'будинок'), ('.', '.')]



In [90]:
for k, v in res.items():
    if "електро" in k.lower():
        print(v, k)

565 Відсутнє електропостачання
553 Ремонт та заміна електроприладів
514 Перебої електропостачання


In [91]:
for k, v in res.items():
    if "паркув" in k.lower():
        print(v, k)

616 Паркування авто у місцях загального користування
359 Паркування авто на прибудинковій території
218 Розміщення паркувальних майданчиків
190 Зберігання транспортних засобів, порушення правил паркування
105 Паркування на зеленій зоні у місцях загального користування


In [92]:
for k, v in res.items():
    if "опал" in k.lower():
        print(v, k)

6020 Відсутність опалення
2049 Відсутність опалення по стояку
1341 Незадовільна температура опалення
709 Встановлення лічильників на опалення.
256 Питання, що стосуються завершення опалювального сезону
212 Перерахунок плати за відсутність опалення
160 Опалення на сходових клітинах
154 Питання, що виникають до початку опалювального сезону


In [93]:
for k, v in res.items():
    if "гвп" in k.lower():
        print(v, k)

13273 Відсутність ГВП
2304 Незадовільна температура ГВП
303 Недостатній тиск ГВП
288 Незадовільна робота рушникосушарок від ГВП
226 Перерахунок плати за відсутність ГВП
163 Перерахунок плати за незадовільну температуру ГВП
114 Відсутність ГВП по стояку


In [94]:
for k, v in res.items():
    if "хвп" in k.lower():
        print(v, k)

1562 Відсутнє ХВП
349 Відсутній тиск ХВП з 10-го поверху включно (”Київенерго”)
233 Відсутній тиск ХВП до 9-го поверху включно (”Київводоканал” )
194 Неякісне ХВП
